In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(database_name):
    conn = None
    try : 
        conn = sqlite3.connect(database_name)
        print(f"Conexão bem sucedidda com sqlite {sqlite3.version}")
    except Error as e:
        print(f'Error {e} na conexão')
    return conn
    

In [3]:
def execute_query(conn,query):
    try:
        df = pd.read_sql(query,conn)
        display(df)
    except Error as e:
        print(f'Erro {e} na execução de consultas')

In [4]:
conn = create_connection('extra_practice_database.db')
with conn:
    print("Query 1 :  Liste todos os clientes junto com o total acumulado que cada um gastou")
    query1 = """ --- Criando uma tabela com a venda acumulada por customerid
    WITH venda_acumulada_por_customerid
     AS (SELECT customerid,
                Sum(total)
                  OVER (
                    partition BY customerid) AS total_acumulado
         FROM   invoice)
SELECT b.firstname
       || ""
       || b.lastname AS nome_completo,
       total_acumulado
FROM   venda_acumulada_por_customerid AS a
       LEFT JOIN customer AS b
              ON a.customerid = b.customerid 

    """
    execute_query(conn,query1) 

Conexão bem sucedidda com sqlite 2.6.0
Query 1 :  Liste todos os clientes junto com o total acumulado que cada um gastou


,nome_completo,total_acumulado
0,LuísGonçalves,39.62
1,LuísGonçalves,39.62
2,LuísGonçalves,39.62
3,LuísGonçalves,39.62
4,LuísGonçalves,39.62
...,...,...
407,PujaSrivastava,36.64
408,PujaSrivastava,36.64
409,PujaSrivastava,36.64
410,PujaSrivastava,36.64


In [5]:
with conn:
    print("Query 2 :  Para cada faixa (track), mostre o nome da faixa e o preço médio de todas as faixas do mesmo gênero.")
    query2 = """
    SELECT NAME,
       Avg (unitprice)
         OVER ( -- Over é a função que abre a janela
           PARTITION BY genreid) AS media_por_genero -- partition By eu leio como particionando po
FROM track 

    """
    execute_query(conn,query2) 

Query 2 :  Para cada faixa (track), mostre o nome da faixa e o preço médio de todas as faixas do mesmo gênero.


,Name,media_por_genero
0,For Those About To Rock (We Salute You),0.99
1,Balls to the Wall,0.99
2,Fast As a Shark,0.99
3,Restless and Wild,0.99
4,Princess of the Dawn,0.99
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,0.99
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",0.99
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",0.99
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",0.99


In [7]:

with conn:
    print("Query 3 :  Liste todos os álbuns junto com o número total de faixas em cada álbum")
    query3 = """
    SELECT b.title,
       COUNT(trackid)
         OVER (
           partition BY a.albumid) AS qt_faixas_no_album
FROM   track AS a
       LEFT JOIN album AS b
              ON a.albumid = b.albumid 


    """
    execute_query(conn,query3) 

Query 3 :  Liste todos os álbuns junto com o número total de faixas em cada álbum


,Title,qt_faixas_no_album
0,For Those About To Rock We Salute You,10
1,For Those About To Rock We Salute You,10
2,For Those About To Rock We Salute You,10
3,For Those About To Rock We Salute You,10
4,For Those About To Rock We Salute You,10
...,...,...
3498,Respighi:Pines of Rome,1
3499,Schubert: The Late String Quartets & String Qu...,1
3500,Monteverdi: L'Orfeo,1
3501,Mozart: Chamber Music,1


In [8]:
with conn:
    print("Query 4 :  Liste todos os clientes e mostre o total gasto por cada um em suas compras, junto com o total acumulado gasto até aquela compra específica")
    query4 = """
WITH venda_acumulada_por_customerid
     AS (SELECT customerid,
                invoicedate,
                total,
                Sum(total)
                  OVER (
                    partition BY customerid -- Usamos o partition para particionar e order para ir acumulando até a presente data
                    ORDER BY invoicedate) AS total_acumulado
         FROM   invoice)
SELECT b.firstname
       || ""
       || b.lastname AS nome_completo,
       a.invoicedate,
       a.total,
       total_acumulado
FROM   venda_acumulada_por_customerid AS a
       LEFT JOIN customer AS b
              ON a.customerid = b.customerid 

    """
    execute_query(conn,query4) 

Query 4 :  Liste todos os clientes e mostre o total gasto por cada um em suas compras, junto com o total acumulado gasto até aquela compra específica


,nome_completo,invoicedate,total,total_acumulado
0,LuísGonçalves,2010-03-11 00:00:00,3.98,3.98
1,LuísGonçalves,2010-06-13 00:00:00,3.96,7.94
2,LuísGonçalves,2010-09-15 00:00:00,5.94,13.88
3,LuísGonçalves,2011-05-06 00:00:00,0.99,14.87
4,LuísGonçalves,2012-10-27 00:00:00,1.98,16.85
...,...,...,...,...
407,PujaSrivastava,2009-07-08 00:00:00,5.94,9.90
408,PujaSrivastava,2010-02-26 00:00:00,1.99,11.89
409,PujaSrivastava,2011-08-20 00:00:00,1.98,13.87
410,PujaSrivastava,2011-09-30 00:00:00,13.86,27.73


In [9]:
with conn:
    print("Query 5 :  Para cada funcionário, mostre o título do funcionário, a data de contratação e a classificação de antiguidade (rank) em relação à data de contratação.")
    query5 = """
SELECT employeeid,
       title,
       hiredate,
       RANK () -- Função que junta dois valores no mesmo ranking e o posterior vai virar a posição repetida + n (vezes que se repetiu)
         OVER (
           ORDER BY hiredate) AS antiguidade_rank,
       Dense_rank() -- Funçao que junta dois valores no mesmo ranking e o posterior vai virar a posição repetida + 1
         OVER (
           ORDER BY hiredate) AS antiguidade_dense_rank, 
       Row_number() -- Função que atribui valores diferentes para itens de mesma posição considerando a posição que aparece
         OVER (
           ORDER BY hiredate) AS antiguidade_row_number
FROM   employee 

    """
    execute_query(conn,query5) 

Query 5 :  Para cada funcionário, mostre o título do funcionário, a data de contratação e a classificação de antiguidade (rank) em relação à data de contratação.


,EmployeeId,Title,HireDate,antiguidade_rank,antiguidade_dense_rank,antiguidade_row_number
0,3,Sales Support Agent,2002-04-01 00:00:00,1,1,1
1,2,Sales Manager,2002-05-01 00:00:00,2,2,2
2,1,General Manager,2002-08-14 00:00:00,3,3,3
3,4,Sales Support Agent,2003-05-03 00:00:00,4,4,4
4,5,Sales Support Agent,2003-10-17 00:00:00,5,5,5
5,6,IT Manager,2003-10-17 00:00:00,5,5,6
6,7,IT Staff,2004-01-02 00:00:00,7,6,7
7,8,IT Staff,2004-03-04 00:00:00,8,7,8


In [17]:
with conn:
    print("Query 6 :Para cada cliente, liste todas as compras realizadas e a diferença de tempo (em dias) entre cada compra e a compra anterior.")
    query6 = """
    WITH CompraAnterior AS ( 
        SELECT invoicedate,
           Customerid,
           LAG(invoicedate, 1) OVER (PARTITION BY Customerid) AS DtCompraAnterior
      FROM invoice
),
diff_dias_por_customerid AS (
    SELECT customerid,
           julianday(invoicedate) - julianday(DtCompraAnterior) AS diff_entre_duas_compras_em_dias
      FROM CompraAnterior
     WHERE DtCompraAnterior IS NOT NULL
)
SELECT b.firstname || " " || b.lastname AS nome_completo,
       CAST(a.diff_entre_duas_compras_em_dias AS INT) AS diferenca_compras_sucessivas_dias
  FROM diff_dias_por_customerid AS a
       LEFT JOIN
       customer AS b ON a.customerid = b.customerid;
    """
    execute_query(conn,query6) 

Query 6 :Para cada cliente, liste todas as compras realizadas e a diferença de tempo (em dias) entre cada compra e a compra anterior.


,nome_completo,diferenca_compras_sucessivas_dias
0,Luís Gonçalves,94
1,Luís Gonçalves,94
2,Luís Gonçalves,233
3,Luís Gonçalves,540
4,Luís Gonçalves,41
...,...,...
348,Puja Srivastava,94
349,Puja Srivastava,233
350,Puja Srivastava,540
351,Puja Srivastava,41


In [16]:
with conn:
    print("Query 7 : Para cada funcionário, liste o número total de funcionários que foram contratados antes dele.")
    query7 = """
       SELECT firstname,
       lastname,
       title,
       hiredate,
       ROW_NUMBER()
         OVER (
           ORDER BY hiredate) - 1 AS qt_funcionarios_contratados_antes
-- subtraindo de 1 para saber quantos clientes foram contratados, pois ele mesmo nao conta
FROM   employee 
    """
    execute_query(conn,query7) 

Query 7 : Para cada funcionário, liste o número total de funcionários que foram contratados antes dele.


,FirstName,LastName,Title,HireDate,qt_funcionarios_contratados_antes
0,Jane,Peacock,Sales Support Agent,2002-04-01 00:00:00,0
1,Nancy,Edwards,Sales Manager,2002-05-01 00:00:00,1
2,Andrew,Adams,General Manager,2002-08-14 00:00:00,2
3,Margaret,Park,Sales Support Agent,2003-05-03 00:00:00,3
4,Steve,Johnson,Sales Support Agent,2003-10-17 00:00:00,4
5,Michael,Mitchell,IT Manager,2003-10-17 00:00:00,5
6,Robert,King,IT Staff,2004-01-02 00:00:00,6
7,Laura,Callahan,IT Staff,2004-03-04 00:00:00,7
